In [ ]:
import sys
import os
import time

import pandas as pd
import numpy as np
import json as json
import statistics as stat
import sklearn.metrics as skmetrics
import sklearn.model_selection as skms

import pymetry as pym
import pymetry.ds as pym_ds
import pymetry.project as pym_prj
import pymetry.utilities.jobs as pym_jobs
import pymetry.utilities.utils as pym_ut
import pymetry.explore as pym_exp
import pymetry.federation as fed

### Federation & peer names

In [ ]:
fed_admin_name = "fed_name_iris_01"

# Peer 0 is the Federation Administrator
peer0_name = "peer0"
peer1_name = "peer1"
peer2_name = "peer2"

### Read config information

In [ ]:
pym.init_conf({
    "server" : 'http://YOUR_IP_ADDRESS',
    "sym_key_id" : "YOUR_SYMETRY_KEYID",
    "sym_secret_key" : "YOUR_SYMETRY_SECRET_KEY",
    
    "s3accesskey" : "YOUR_AWS_ACCESS_KEY",
    "s3secretkey" : "YOUR_AWS_S3_SECRET_KEY",
    
    "fedadmin_password" : "YOUR_FEDERATION_PASSWORD",
    "fedadmin_s3_bucket": "YOUR_S3_BUCKET",
    "fedadminuser" : "YOUR_AWS_ACCOUNTID",
    
    "fedpeer1_S3_bucket" : "YOUR_S3_BUCKET",
    "fedpeer1user" : "YOUR_AWS_ACCOUNTID",
    
    "fedpeer2_S3_bucket" : "YOUR_S3_BUCKET",
    "fedpeer2user" : "YOUR_AWS_ACCOUNTID"
})

## Read the data

### Iris Dataset

In [ ]:
DATA_FILE = "../../../data/Iris_data.csv"

df = pd.read_csv(DATA_FILE)

# Dropping Columns to avoid Singular Matrix Error
columns_drop = ['sepal_lengt_b2',
                'sepal_width_b2',
                'petal_length_b2',
                'petal_width_b2',
                'Iris_versicolor',
                'Iris_virginica',
                'Iris_setosa']

df = df.drop(columns_drop, 
             axis=1)

# Get list of Attributes & Target
attributes_target = df.columns.tolist()

### Shuffle Dataset & separate into 3 equal parts for each peer of the federation
### Create Dataframes for all 3 peers

In [ ]:
shuffled = df.sample(frac=1)
df_out = np.array_split(shuffled, 3) 

In [ ]:
def create_train_test(in_df):
    
    train_df, test_df = skms.train_test_split(in_df, 
                                              test_size=0.2, 
                                              random_state=42)

    print(train_df.shape)
    print(test_df.shape)

    smldf_train = pym_ut.pandas_df_to_sml_json(pandas_df=train_df)
    smldf_test = pym_ut.pandas_df_to_sml_json(pandas_df=test_df)
    
    rval = (smldf_train, 
            smldf_test, 
            test_df)
    
    return rval

In [ ]:
smldf0_train, smldf0_test, df0_test = create_train_test(df_out[0])
smldf1_train, smldf1_test, df1_test = create_train_test(df_out[1])
smldf2_train, smldf2_test, df2_test = create_train_test(df_out[2])

### Creating Federation Admin

In [ ]:
def create_fedadmin_info(in_fed_admin_name,
                         in_fed_peer0_name):
    
    fed_name_tmp = in_fed_admin_name
    fed_local_tmp = in_fed_peer0_name
    fed_local_type_tmp = 'cpu'
    aws_access_key_tmp = pym.conf['s3accesskey']
    aws_secret_key_tmp = pym.conf['s3secretkey']
    aws_region_tmp = 'US_EAST_1'
    s3_bucket_tmp = pym.conf['fedadmin_s3_bucket']
    aws_id_tmp = pym.conf['fedadminuser']
    sync_sched_tmp = 'm1'

    fed_info_out = fed.get_fed_info(fed_name_tmp,
                                    fed_local_tmp,
                                    fed_local_type_tmp,
                                    aws_access_key_tmp,
                                    aws_secret_key_tmp,
                                    aws_region_tmp,
                                    s3_bucket_tmp,
                                    aws_id_tmp,
                                    sync_sched_tmp)
    rval = fed_info_out

    return rval

In [ ]:
fed_info_out = create_fedadmin_info(fed_admin_name,
                                    peer0_name)

### Creating Federation Peer & Learning training data

In [ ]:
fed_info = fed.create_federation(peer0_name,
                                 fed_info_out)
print(fed_info.content)
histo = pym_prj.enable_histogram(peer0_name)

In [ ]:
pro_learnls = pym_prj.stream_data_to_project(peer0_name, 
                                             smldf0_train, 
                                             client_id='pymetry')
print(pro_learnls.content)

In [ ]:
def fed_join_created(in_fed_encrypt_passwd,
                     in_fed_join_name,
                     in_peer_userid,
                     in_peer_user_s3bucket,
                     in_peer_s3accesskey,
                     in_peer_s3secretkey):
    
    join_local_name_in = in_fed_join_name
    join_local_type_in = 'cpu'
    join_aws_id_in = in_peer_userid
    join_s3_bucket_in =  in_peer_user_s3bucket
    join_access_key_in = in_peer_s3accesskey
    join_secret_key_in = in_peer_s3secretkey
    join_aws_region_in = 'US_EAST_1'

    fed_join_body = fed.get_fed_join_info(in_fed_encrypt_passwd,
                                          join_local_name_in,
                                          join_local_type_in,
                                          join_aws_id_in,
                                          join_s3_bucket_in,
                                          join_access_key_in,
                                          join_secret_key_in,
                                          join_aws_region_in)
    rval = fed_join_body
    return rval

In [ ]:
def fed_encrypt_create(in_fed_peer0_name,
                       in_fed_peerx_name,
                       in_peer_userid,
                       in_peer_user_s3bucket,
                       in_peer_s3accesskey,
                       in_peer_s3secretkey):

    fed_encrypt_passwd = pym.conf['fedadmin_password']
    fed_encrypt_rest_host = pym.conf['server']

    fed_pad_passwd = fed.pad_password(fed_encrypt_passwd)

    fed_encrypt_info_out = fed.get_fed_encrypt_info(fed_encrypt_passwd,
                                                    fed_encrypt_rest_host)
    
    fed_encrypt_out = fed.get_fed_encrypt(in_fed_peer0_name, 
                                          fed_encrypt_info_out)

    fed_msg_decrypt = fed_encrypt_out.value['encryptedGroupInfo']

    decrypt_output_tmp = fed.decrypt_fed_info(fed_msg_decrypt, 
                                              fed_pad_passwd)

    fed_join_body_tmp = fed_join_created(fed_encrypt_passwd,
                                         in_fed_peerx_name,
                                         in_peer_userid,
                                         in_peer_user_s3bucket,
                                         in_peer_s3accesskey,
                                         in_peer_s3secretkey)

    fed_json = json.loads(decrypt_output_tmp)

    fed_json.update(fed_join_body_tmp)

    rval = fed_json

    return rval

In [ ]:
def join_fed_learn_data(in_fed_peer0_name,
                        in_fed_peerx_name,
                        in_smldf_train,
                        in_peer_userid,
                        in_peer_user_s3bucket,
                        in_peer_s3accesskey,
                        in_peer_s3secretkey):
    
    encrypt_out = fed_encrypt_create(in_fed_peer0_name,
                                     in_fed_peerx_name,
                                     in_peer_userid,
                                     in_peer_user_s3bucket,
                                     in_peer_s3accesskey,
                                     in_peer_s3secretkey)
                                

    join_status = fed.create_join_federation(in_fed_peerx_name, 
                                             encrypt_out)
    
    pym_prj.enable_histogram(in_fed_peerx_name)
    
    print(in_fed_peerx_name + ": %s " % join_status.content)
    time.sleep(2)
    
    pro_learnls = pym_prj.stream_data_to_project(in_fed_peerx_name, 
                                                 in_smldf_train, 
                                                 client_id='pymetry')

### Peer 1 joining Federation

In [ ]:
peer1_userid = pym.conf['fedpeer1user']
peer1_s3bucket = pym.conf['fedpeer1_s3_bucket']
peer1_accesskey = pym.conf['s3accesskey']
peer1_secretkey = pym.conf['s3secretkey']

# import ipdb; ipdb.set_trace()

join_fed_learn_data(peer0_name,
                    peer1_name,
                    smldf1_train,
                    peer1_userid,
                    peer1_s3bucket,
                    peer1_accesskey,
                    peer1_secretkey)

### Peer 2 joining Federation

In [ ]:
peer2_userid = pym.conf['fedpeer2user']
peer2_s3bucket = pym.conf['fedpeer2_s3_bucket']
peer2_accesskey = pym.conf['s3accesskey']
peer2_secretkey = pym.conf['s3secretkey']

join_fed_learn_data(peer0_name,
                    peer2_name,
                    smldf2_train,
                    peer2_userid,
                    peer2_s3bucket,
                    peer2_accesskey,
                    peer2_secretkey)

### Start Pulsing

In [ ]:
time.sleep(5)
pulse_out = fed.start_pulse(peer0_name)
print(peer0_name + ": %s " % pulse_out.content)    
    
pulse_out = fed.start_pulse(peer1_name)
print(peer1_name + ": %s " % pulse_out.content)

pulse_out = fed.start_pulse(peer2_name)
print(peer2_name + ": %s " % pulse_out.content)

### Pause for 5 minutes to let all Peers Sync information

In [ ]:
pause_time = 300

print("Pausing for %s seconds to allow Federation to sync information\
for all peers " % pause_time)
time.sleep(pause_time)

### Monitor Federation Peers Sync & Error Logs

In [ ]:
def get_synclogs(in_project):
    
    rsp = fed.get_fed_sync_log(in_project)
    sync_peer = rsp.json['values']['stringList']['values']
    for i in range(0, len(sync_peer)):
        print(sync_peer[i])

In [ ]:
def get_errorlogs(in_project):
    
    rsp = fed.get_fed_error_log(in_project)
    err_peer = rsp.json['values']['stringList']['values']
    for i in range(0, len(err_peer)):
        print(err_peer[i])

#### Peer 0 (Admin) : Sync Logs

In [ ]:
get_synclogs(peer0_name)

#### Peer 0 (Admin) : Error Logs

In [ ]:
get_errorlogs(peer0_name)

#### Peer 1: Sync Logs

In [ ]:
get_synclogs(peer1_name)

#### Peer 1: Error Logs

In [ ]:
get_errorlogs(peer1_name)

#### Peer 2 : Sync Logs

In [ ]:
get_synclogs(peer2_name)

#### Peer 2 : Error Logs

In [ ]:
get_errorlogs(peer2_name)

### Stop Pulsing

In [ ]:
time.sleep(5)
 
pulse_out = fed.stop_pulse(peer0_name)
print(peer0_name + ": %s " % pulse_out.content)

pulse_out = fed.stop_pulse(peer1_name)
print(peer1_name + ": %s " % pulse_out.content)

pulse_out = fed.stop_pulse(peer2_name)
print(peer2_name + ": %s " % pulse_out.content)

### Build Regression Models on all 3 peers

In [ ]:
def build_models(project_name):   
    
    model_name = 'regression_one'
    prj_name = project_name
    ml_context = {"targets": ["0"],
                  "inputAttributes": ["1", "2", "3", "4", "5", "6", "7"]}

    rsp = pym_prj.build_model(ml_context=ml_context,
                              pid=prj_name,
                              modelid=model_name,
                              algo='mlr')
    print(rsp.content)
    pym_jobs.wait_for_job_finish(rsp)

#### Peer 0 (Admin) : Build Model 

In [ ]:
build_models(peer0_name)

#### Peer 1: Build Model

In [ ]:
build_models(peer1_name)

#### Peer 2: Build Model

In [ ]:
build_models(peer2_name)

### Make Predictions

In [ ]:
def predict(peer_name_in,
            model_name_in,
            smldf_test_in):

    rsp = pym_prj.predict_df(peer_name_in,
                             model_name_in,
                             smldf_test_in)
    print(rsp.content)
    
    rst = rsp.json['values']['KSVSMap']['values']
    pred_values = [ ]
    for i in range(0,len(rst)):
        tt = list(rst[i].values())
        pred_values.append(float(tt[0]))
    rval = pred_values
    return rval

#### Peer 0 (Admin) : Prediction

In [ ]:
model_name = 'regression_one'

y_pred0 = predict(peer0_name,
                  model_name,
                  smldf0_test)

#### Peer 1 : Prediction

In [ ]:
y_pred1 = predict(peer1_name,
                  model_name,
                  smldf1_test)

#### Peer 2 : Prediction

In [ ]:
y_pred2 = predict(peer2_name,
                  model_name,
                  smldf2_test)

### R Square Measure

#### Peer 0 (Admin) : R Square

In [ ]:
y_true0 = df0_test['sepal_length'].values.tolist()
rsquare0 = skmetrics.r2_score(y_true0, 
                              y_pred0)
print("R Square for peer0 : %s " % rsquare0)

#### Peer 1 : R Square

In [ ]:
y_true1 = df1_test['sepal_length'].values.tolist()
rsquare1 = skmetrics.r2_score(y_true1, 
                              y_pred1)
print("R Square for peer1 : %s " % rsquare1)

#### Peer 2 : R Square

In [ ]:
y_true2 = df2_test['sepal_length'].values.tolist()
rsquare2 = skmetrics.r2_score(y_true2, 
                              y_pred2)
print("R Square for peer2 : %s " % rsquare2)